In [8]:
%run /Users/denis/Documents/Projects/scripts/init.ipy
os.chdir('..')

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [157]:
infiles = ['03-articles/bioinformatics_articles.txt', '03-articles/bmc-bioinformatics_articles.txt', '03-articles/database_articles.txt', '03-articles/nar_articles.txt']

In [161]:
# Initialize dataframe
abstract_dataframe = pd.concat([pd.read_table(x)[['abstract', 'doi']] for x in infiles])

# Fix abstract
abstract_dataframe['abstract'] = [' '.join([abstract_tuple[1] for abstract_tuple in json.loads(x)['abstract'] if abstract_tuple[0] not in [u'Contact:', u'Availability and implementation', u'Supplementary information']]) for x in abstract_dataframe['abstract']]

# Get dict
abstract_dict = abstract_dataframe.set_index('doi').to_dict()['abstract']

# Get vectorizer
vect = TfidfVectorizer(min_df=1)

# Compute similarity
tfidf = vect.fit_transform(abstract_dict.values())

# Get similarity dataframe
similarity_dataframe = pd.melt(pd.DataFrame((tfidf * tfidf.T).A, columns=abstract_dict.keys(), index=abstract_dict.keys()).reset_index(), id_vars='index').rename(columns={'index': 'source_abstract', 'variable': 'target_abstract'})

In [166]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df=1)
tfidf = vect.fit_transform(abstract_dict.values())

In [173]:
# Get similarity dataframe
similarity_dataframe = pd.melt(pd.DataFrame((tfidf * tfidf.T).A, columns=abstract_dict.keys(), index=abstract_dict.keys()).reset_index(), id_vars='index').rename(columns={'index': 'source_abstract', 'variable': 'target_abstract'})

,source_abstract,target_abstract,value
0,https://doi.org/10.1093/bioinformatics/btu599,https://doi.org/10.1093/bioinformatics/btu599,1.000000
1,https://doi.org/10.1093/database/baw006,https://doi.org/10.1093/bioinformatics/btu599,0.091130
2,https://doi.org/10.1093/bioinformatics/btu590,https://doi.org/10.1093/bioinformatics/btu599,0.076575
3,https://doi.org/10.1093/bioinformatics/btu593,https://doi.org/10.1093/bioinformatics/btu599,0.040230
4,https://doi.org/10.1093/bioinformatics/btu595,https://doi.org/10.1093/bioinformatics/btu599,0.064747
5,https://doi.org/10.1093/bioinformatics/btu594,https://doi.org/10.1093/bioinformatics/btu599,0.102219
6,https://doi.org/10.1093/bioinformatics/btu596,https://doi.org/10.1093/bioinformatics/btu599,0.066307
7,https://doi.org/10.1093/bioinformatics/btw557,https://doi.org/10.1093/bioinformatics/btu599,0.093690
8,https://doi.org/10.1093/bioinformatics/btw554,https://doi.org/10.1093/bioinformatics/btu599,0.082272
9,https://doi.org/10.1093/nar/gkw925,https://doi.org/10.1093/bioinformatics/btu599,0.108333
